<a href="https://colab.research.google.com/github/April27Dev/Motor-Imagery-Classification/blob/main/Motor3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install necessary libraries
!pip install mne numpy pandas scikit-learn tensorflow

# Step 2: Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization, Dropout, Activation
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, cohen_kappa_score
import mne
from google.colab import files

# Step 3: Upload the dataset
uploaded = files.upload()
data_path = list(uploaded.keys())[0]  # Get the uploaded file name
data = pd.read_csv(data_path)

# Step 4: Filter for 'left' and 'right' labels
data = data[data['label'].isin(['left', 'right'])]

# Map labels to binary values (0 for left, 1 for right)
data['label'] = data['label'].map({'left': 0, 'right': 1})

# Extract EEG data and labels
eeg_channels = [col for col in data.columns if "EEG" in col]
X = data[eeg_channels].values  # EEG features
y = data['label'].values  # Labels

# Reshape EEG data into trials (assuming 22 channels and sampling rate of 250 Hz)
# Determine the number of samples per trial based on unique epochs
epochs = data['epoch'].unique()
samples_per_epoch = data[data['epoch'] == epochs[0]].shape[0]
num_trials = len(epochs)

# Reshape data into (trials, channels, time, 1) for CNN input
X = X.reshape(num_trials, 22, samples_per_epoch, 1)

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

# Convert labels to categorical format
y = to_categorical(y[:num_trials])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Define the CNN model
model = Sequential()

# Temporal convolutional layers
model.add(Conv2D(50, (1, 8), strides=(1, 6), input_shape=(22, samples_per_epoch, 1), padding="valid"))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(50, (1, 16), strides=(1, 6), padding="valid"))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Spatial convolution layer
model.add(Conv2D(20, (22, 1), strides=(1, 1), padding="valid"))
model.add(BatchNormalization())
model.add(Activation('relu'))

# Pooling Layer Adjustment
if samples_per_epoch >= 5:
    model.add(AveragePooling2D(pool_size=(1, 2), strides=(1, 1)))
else:
    print("Skipping pooling due to insufficient time dimension.")

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))  # Binary classification: left vs. right

# Step 6: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Step 7: Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=64, verbose=1)

# Step 8: Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))
print("Cohen's Kappa:", cohen_kappa_score(y_true, y_pred))


Saving BCICIV_2a_1.csv to BCICIV_2a_1 (1).csv


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 523ms/step - accuracy: 0.3920 - loss: 0.8944 - val_accuracy: 1.0000 - val_loss: 0.6573
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8745 - loss: 0.4631 - val_accuracy: 1.0000 - val_loss: 0.6181
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9626 - loss: 0.2963 - val_accuracy: 1.0000 - val_loss: 0.5815
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9875 - loss: 0.2244 - val_accuracy: 1.0000 - val_loss: 0.5479
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 1.0000 - loss: 0.1382 - val_accuracy: 1.0000 - val_loss: 0.5178
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 1.0000 - loss: 0.1072 - val_accuracy: 1.0000 - val_loss: 0.4903
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 1.0000 - loss: 0.0904 - val_accuracy: 1.0000 - val_loss: 0.4648
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 0.0474 - val_accuracy: 1.0000 - val_loss: 0.4409

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
